# Lesson A6: Macros & Metaprogramming

**Duration**: 120-150 minutes  
**Stage**: Advanced (Mastery)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Create declarative macros with macro_rules!
2. Understand macro hygiene and scoping
3. Build procedural macros (conceptually)
4. Apply macros for code generation and DSLs
5. Debug and test macro code effectively

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do async functions work with the Future trait?
2. What's the difference between async and threaded concurrency?
3. How do you handle errors in async code?
4. What are the benefits of async I/O?

**Answers**: 1) Return impl Future, executed by runtime, 2) Async is cooperative, threads are preemptive, 3) Result types with ? operator, 4) Non-blocking, high concurrency, efficient resource usage

---

## 🧠 Key Concepts

### Macro Types

**Declarative Macros**: Pattern-based code generation with macro_rules!
**Procedural Macros**: Custom derive, attribute, and function-like macros
**Macro Hygiene**: Automatic variable scoping to prevent conflicts
**Token Trees**: How Rust parses and represents macro input

### Metaprogramming Benefits

- **Code Generation**: Reduce boilerplate and repetition
- **Domain-Specific Languages**: Create expressive APIs
- **Compile-Time Computation**: Zero-runtime-cost abstractions
- **Type Safety**: Generate type-safe code automatically

---

## 🔬 Live Code Exploration

### Basic Declarative Macros

In [ ]:
// Basic declarative macros with macro_rules!

// Simple function-like macro
macro_rules! say_hello {
    () => {
        println!("Hello from macro!");
    };
}

// Macro with parameters
macro_rules! greet {
    ($name:expr) => {
        println!("Hello, {}!", $name);
    };
}

// Macro with multiple patterns
macro_rules! calculate {
    // Addition
    ($a:expr + $b:expr) => {
        $a + $b
    };
    // Multiplication
    ($a:expr * $b:expr) => {
        $a * $b
    };
    // Power (using repetition)
    ($base:expr ^ $exp:expr) => {
        (0..$exp).fold(1, |acc, _| acc * $base)
    };
}

// Macro with repetition
macro_rules! create_function {
    ($func_name:ident, $($param:ident: $param_type:ty),*) => {
        fn $func_name($($param: $param_type),*) {
            println!("Function {} called with parameters:", stringify!($func_name));
            $(
                println!("  {}: {:?}", stringify!($param), $param);
            )*
        }
    };
}

// Macro for creating structs with debug info
macro_rules! debug_struct {
    ($name:ident { $($field:ident: $field_type:ty),* $(,)? }) => {
        #[derive(Debug, Clone)]
        struct $name {
            $($field: $field_type),*
        }
        
        impl $name {
            fn new($($field: $field_type),*) -> Self {
                $name {
                    $($field),*
                }
            }
            
            fn describe(&self) {
                println!("Struct {}: {:?}", stringify!($name), self);
                $(
                    println!("  {}: {:?}", stringify!($field), self.$field);
                )*
            }
        }
    };
}

// Macro for creating enums with utility methods
macro_rules! smart_enum {
    ($name:ident { $($variant:ident),* $(,)? }) => {
        #[derive(Debug, Clone, PartialEq)]
        enum $name {
            $($variant),*
        }
        
        impl $name {
            fn all_variants() -> Vec<$name> {
                vec![$($name::$variant),*]
            }
            
            fn variant_count() -> usize {
                let mut count = 0;
                $(count += 1; let _ = $name::$variant;)*
                count
            }
            
            fn name(&self) -> &'static str {
                match self {
                    $($name::$variant => stringify!($variant)),*
                }
            }
        }
    };
}

fn basic_macros_demo() {
    println!("=== Basic Declarative Macros ===");
    
    // Simple macros
    say_hello!();
    greet!("Rust Developer");
    
    // Calculation macro
    println!("5 + 3 = {}", calculate!(5 + 3));
    println!("4 * 7 = {}", calculate!(4 * 7));
    println!("2 ^ 5 = {}", calculate!(2 ^ 5));
    
    // Generated function
    create_function!(test_func, x: i32, y: String, z: bool);
    test_func(42, "hello".to_string(), true);
    
    // Generated struct
    debug_struct!(Person {
        name: String,
        age: u32,
        active: bool,
    });
    
    let person = Person::new("Alice".to_string(), 30, true);
    person.describe();
    
    // Generated enum
    smart_enum!(Color {
        Red,
        Green,
        Blue,
        Yellow,
    });
    
    println!("\nColor enum has {} variants", Color::variant_count());
    println!("All colors: {:?}", Color::all_variants());
    
    let color = Color::Blue;
    println!("Selected color: {} ({:?})", color.name(), color);
}

basic_macros_demo();

### Advanced Macro Patterns

In [ ]:
// Advanced macro patterns and techniques

// Macro for creating a hash map with syntax sugar
macro_rules! hashmap {
    // Empty map
    () => {
        std::collections::HashMap::new()
    };
    // Map with key-value pairs
    ($($key:expr => $value:expr),* $(,)?) => {
        {
            let mut map = std::collections::HashMap::new();
            $(
                map.insert($key, $value);
            )*
            map
        }
    };
}

// Macro for creating a builder pattern
macro_rules! builder {
    ($name:ident {
        $($field:ident: $field_type:ty),* $(,)?
    }) => {
        #[derive(Debug, Default)]
        struct $name {
            $($field: Option<$field_type>),*
        }
        
        paste::paste! {
            impl $name {
                fn new() -> Self {
                    Self::default()
                }
                
                $(
                    fn [<with_ $field>](mut self, $field: $field_type) -> Self {
                        self.$field = Some($field);
                        self
                    }
                    
                    fn [<set_ $field>](&mut self, $field: $field_type) {
                        self.$field = Some($field);
                    }
                    
                    fn [<get_ $field>](&self) -> Option<&$field_type> {
                        self.$field.as_ref()
                    }
                )*
                
                fn build(self) -> Result<[<$name Built>], String> {
                    $(
                        let $field = self.$field.ok_or_else(|| {
                            format!("Missing required field: {}", stringify!($field))
                        })?;
                    )*
                    
                    Ok([<$name Built>] {
                        $($field),*
                    })
                }
            }
        }
        
        paste::paste! {
            #[derive(Debug)]
            struct [<$name Built>] {
                $($field: $field_type),*
            }
        }
    };
}

// Note: The paste crate is not available in evcxr, so let's create a simpler version
macro_rules! simple_builder {
    ($name:ident {
        $($field:ident: $field_type:ty),* $(,)?
    }) => {
        #[derive(Debug, Default)]
        struct $name {
            $($field: Option<$field_type>),*
        }
        
        impl $name {
            fn new() -> Self {
                Self::default()
            }
            
            $(
                fn $field(mut self, value: $field_type) -> Self {
                    self.$field = Some(value);
                    self
                }
            )*
            
            fn build(self) -> Result<String, String> {
                let mut result = format!("{}Builder {{", stringify!($name));
                $(
                    if let Some(value) = self.$field {
                        result.push_str(&format!(" {}: {:?},", stringify!($field), value));
                    } else {
                        return Err(format!("Missing field: {}", stringify!($field)));
                    }
                )*
                result.push_str(" }");
                Ok(result)
            }
        }
    };
}

// Macro for creating test cases
macro_rules! test_cases {
    ($test_name:ident: $($case:expr => $expected:expr),* $(,)?) => {
        fn $test_name() {
            println!("Running test: {}", stringify!($test_name));
            let mut passed = 0;
            let mut total = 0;
            
            $(
                total += 1;
                let result = $case;
                if result == $expected {
                    println!("  ✅ Case {}: {} == {} (PASS)", total, result, $expected);
                    passed += 1;
                } else {
                    println!("  ❌ Case {}: {} != {} (FAIL)", total, result, $expected);
                }
            )*
            
            println!("  Result: {}/{} tests passed\n", passed, total);
        }
    };
}

// Macro for creating a simple DSL
macro_rules! query {
    (SELECT $($field:ident),* FROM $table:ident WHERE $condition:expr) => {
        {
            let fields = vec![$(stringify!($field)),*];
            let table = stringify!($table);
            let condition = stringify!($condition);
            
            format!("SQL Query: SELECT {} FROM {} WHERE {}", 
                   fields.join(", "), table, condition)
        }
    };
    (INSERT INTO $table:ident VALUES ($($value:expr),*)) => {
        {
            let table = stringify!($table);
            let values = vec![$(format!("{:?}", $value)),*];
            
            format!("SQL Query: INSERT INTO {} VALUES ({})", 
                   table, values.join(", "))
        }
    };
}

// Macro for logging with levels
macro_rules! log {
    (DEBUG: $($arg:tt)*) => {
        println!("[DEBUG] {}", format!($($arg)*));
    };
    (INFO: $($arg:tt)*) => {
        println!("[INFO]  {}", format!($($arg)*));
    };
    (WARN: $($arg:tt)*) => {
        println!("[WARN]  {}", format!($($arg)*));
    };
    (ERROR: $($arg:tt)*) => {
        println!("[ERROR] {}", format!($($arg)*));
    };
}

fn advanced_macros_demo() {
    println!("\n=== Advanced Macro Patterns ===");
    
    // HashMap creation
    let map = hashmap!{
        "name" => "Alice",
        "age" => "30",
        "city" => "New York",
    };
    println!("Created map: {:?}", map);
    
    // Builder pattern
    simple_builder!(Config {
        host: String,
        port: u16,
        ssl: bool,
    });
    
    let config = Config::new()
        .host("localhost".to_string())
        .port(8080)
        .ssl(true)
        .build();
    
    match config {
        Ok(config_str) => println!("Built config: {}", config_str),
        Err(e) => println!("Config error: {}", e),
    }
    
    // Test cases
    test_cases!(math_tests:
        2 + 2 => 4,
        3 * 4 => 12,
        10 / 2 => 5,
        5 - 1 => 4,
    );
    
    math_tests();
    
    // DSL for SQL-like queries
    let query1 = query!(SELECT name, age FROM users WHERE age > 18);
    let query2 = query!(INSERT INTO users VALUES ("Bob", 25, "Engineer"));
    
    println!("Generated queries:");
    println!("  {}", query1);
    println!("  {}", query2);
    
    // Logging macro
    log!(DEBUG: "Debug message with value: {}", 42);
    log!(INFO: "Application started successfully");
    log!(WARN: "Low memory warning: {}MB remaining", 128);
    log!(ERROR: "Failed to connect to database: {}", "timeout");
}

advanced_macros_demo();

### Macro Debugging and Best Practices

In [ ]:
// Macro debugging and best practices

// Macro with debugging capabilities
macro_rules! debug_macro {
    ($($input:tt)*) => {
        {
            println!("Macro input: {}", stringify!($($input)*));
            let result = { $($input)* };
            println!("Macro result: {:?}", result);
            result
        }
    };
}

// Macro with error handling
macro_rules! safe_divide {
    ($a:expr, $b:expr) => {
        {
            let dividend = $a;
            let divisor = $b;
            
            if divisor == 0 {
                Err(format!("Division by zero: {} / {}", dividend, divisor))
            } else {
                Ok(dividend / divisor)
            }
        }
    };
}

// Macro with hygiene demonstration
macro_rules! hygiene_test {
    ($x:expr) => {
        {
            let temp = $x * 2; // This 'temp' is hygienic
            temp + 1
        }
    };
}

// Macro for creating documentation
macro_rules! documented_function {
    (
        $(#[doc = $doc:expr])*
        fn $name:ident($($param:ident: $param_type:ty),*) -> $return_type:ty {
            $($body:tt)*
        }
    ) => {
        $(
            #[doc = $doc]
        )*
        fn $name($($param: $param_type),*) -> $return_type {
            println!("Calling function: {}", stringify!($name));
            $($body)*
        }
    };
}

// Macro for performance measurement
macro_rules! time_it {
    ($name:expr, $code:block) => {
        {
            let start = std::time::Instant::now();
            let result = $code;
            let duration = start.elapsed();
            println!("⏱️  {} took: {:?}", $name, duration);
            result
        }
    };
}

// Macro for creating configuration structs
macro_rules! config_struct {
    (
        $name:ident {
            $(
                $field:ident: $field_type:ty = $default:expr,
            )*
        }
    ) => {
        #[derive(Debug, Clone)]
        struct $name {
            $($field: $field_type),*
        }
        
        impl Default for $name {
            fn default() -> Self {
                $name {
                    $($field: $default),*
                }
            }
        }
        
        impl $name {
            fn new() -> Self {
                Self::default()
            }
            
            $(
                fn $field(mut self, value: $field_type) -> Self {
                    self.$field = value;
                    self
                }
            )*
            
            fn validate(&self) -> Result<(), String> {
                // Add validation logic here if needed
                Ok(())
            }
            
            fn summary(&self) {
                println!("Configuration {}:", stringify!($name));
                $(
                    println!("  {}: {:?}", stringify!($field), self.$field);
                )*
            }
        }
    };
}

// Macro for creating state machines
macro_rules! state_machine {
    (
        $name:ident {
            states: [$($state:ident),*],
            initial: $initial:ident,
            transitions: {
                $(
                    $from:ident -> $to:ident on $event:ident,
                )*
            }
        }
    ) => {
        #[derive(Debug, Clone, PartialEq)]
        enum State {
            $($state),*
        }
        
        #[derive(Debug, Clone)]
        enum Event {
            $($event),*
        }
        
        #[derive(Debug)]
        struct $name {
            current_state: State,
        }
        
        impl $name {
            fn new() -> Self {
                $name {
                    current_state: State::$initial,
                }
            }
            
            fn current_state(&self) -> &State {
                &self.current_state
            }
            
            fn handle_event(&mut self, event: Event) -> Result<(), String> {
                let old_state = self.current_state.clone();
                
                match (&self.current_state, &event) {
                    $(
                        (State::$from, Event::$event) => {
                            self.current_state = State::$to;
                            println!("Transition: {:?} -> {:?} on {:?}", 
                                   old_state, self.current_state, event);
                            Ok(())
                        }
                    )*
                    _ => {
                        Err(format!("Invalid transition: {:?} on {:?}", 
                               self.current_state, event))
                    }
                }
            }
        }
    };
}

fn macro_debugging_demo() {
    println!("\n=== Macro Debugging and Best Practices ===");
    
    // Debug macro usage
    let result = debug_macro!(5 + 3 * 2);
    println!("Final result: {}\n", result);
    
    // Safe division
    match safe_divide!(10, 2) {
        Ok(result) => println!("Division result: {}", result),
        Err(e) => println!("Division error: {}", e),
    }
    
    match safe_divide!(10, 0) {
        Ok(result) => println!("Division result: {}", result),
        Err(e) => println!("Division error: {}", e),
    }
    
    // Hygiene test
    let temp = 5; // This won't conflict with macro's temp
    let result = hygiene_test!(temp);
    println!("\nHygiene test: temp = {}, result = {}", temp, result);
    
    // Documented function
    documented_function!(
        #[doc = "Calculates the square of a number"]
        #[doc = "Returns the input multiplied by itself"]
        fn square(x: i32) -> i32 {
            x * x
        }
    );
    
    let squared = square(7);
    println!("Square of 7: {}", squared);
    
    // Performance measurement
    let result = time_it!("Fibonacci calculation", {
        fn fib(n: u32) -> u32 {
            match n {
                0 | 1 => n,
                _ => fib(n - 1) + fib(n - 2),
            }
        }
        fib(20)
    });
    println!("Fibonacci result: {}", result);
    
    // Configuration struct
    config_struct!(ServerConfig {
        host: String = "localhost".to_string(),
        port: u16 = 8080,
        workers: usize = 4,
        debug: bool = false,
    });
    
    let config = ServerConfig::new()
        .host("0.0.0.0".to_string())
        .port(3000)
        .workers(8)
        .debug(true);
    
    config.summary();
    
    // State machine
    state_machine!(TrafficLight {
        states: [Red, Yellow, Green],
        initial: Red,
        transitions: {
            Red -> Green on Go,
            Green -> Yellow on Caution,
            Yellow -> Red on Stop,
        }
    });
    
    let mut light = TrafficLight::new();
    println!("\nTraffic light initial state: {:?}", light.current_state());
    
    let _ = light.handle_event(Event::Go);
    let _ = light.handle_event(Event::Caution);
    let _ = light.handle_event(Event::Stop);
    
    // Try invalid transition
    if let Err(e) = light.handle_event(Event::Caution) {
        println!("Error: {}", e);
    }
}

macro_debugging_demo();

---

## 🎯 Guided Practice

### Exercise 1: Create a Testing Framework Macro

Build a comprehensive testing framework using macros.

In [ ]:
// TODO: Complete the testing framework macro

// Testing framework with multiple assertion types
macro_rules! test_framework {
    (
        suite $suite_name:ident {
            $(
                test $test_name:ident {
                    $($test_body:tt)*
                }
            )*
        }
    ) => {
        mod $suite_name {
            use super::*;
            
            #[derive(Debug)]
            struct TestResult {
                name: &'static str,
                passed: bool,
                message: String,
            }
            
            impl TestResult {
                fn new(name: &'static str, passed: bool, message: String) -> Self {
                    TestResult { name, passed, message }
                }
            }
            
            $(
                fn $test_name() -> TestResult {
                    println!("  Running test: {}", stringify!($test_name));
                    
                    // Capture test execution
                    let result = std::panic::catch_unwind(|| {
                        $($test_body)*
                    });
                    
                    match result {
                        Ok(_) => TestResult::new(
                            stringify!($test_name),
                            true,
                            "Test passed".to_string()
                        ),
                        Err(e) => {
                            let message = if let Some(s) = e.downcast_ref::<&str>() {
                                s.to_string()
                            } else if let Some(s) = e.downcast_ref::<String>() {
                                s.clone()
                            } else {
                                "Test panicked".to_string()
                            };
                            TestResult::new(
                                stringify!($test_name),
                                false,
                                message
                            )
                        }
                    }
                }
            )*
            
            pub fn run_all_tests() {
                println!("\n🧪 Running test suite: {}", stringify!($suite_name));
                
                let mut results = Vec::new();
                
                $(
                    results.push($test_name());
                )*
                
                // Print results
                let total = results.len();
                let passed = results.iter().filter(|r| r.passed).count();
                let failed = total - passed;
                
                println!("\n📊 Test Results:");
                for result in &results {
                    let status = if result.passed { "✅ PASS" } else { "❌ FAIL" };
                    println!("  {} {}: {}", status, result.name, result.message);
                }
                
                println!("\n📈 Summary: {}/{} tests passed ({:.1}%)", 
                        passed, total, (passed as f64 / total as f64) * 100.0);
                
                if failed > 0 {
                    println!("❌ {} test(s) failed", failed);
                } else {
                    println!("🎉 All tests passed!");
                }
            }
        }
    };
}

// Assertion macros for the testing framework
macro_rules! assert_eq_msg {
    ($left:expr, $right:expr) => {
        {
            let left_val = $left;
            let right_val = $right;
            if left_val != right_val {
                panic!("Assertion failed: {} != {} (left: {:?}, right: {:?})", 
                      stringify!($left), stringify!($right), left_val, right_val);
            }
        }
    };
}

macro_rules! assert_true {
    ($expr:expr) => {
        {
            let val = $expr;
            if !val {
                panic!("Assertion failed: {} is not true (value: {:?})", 
                      stringify!($expr), val);
            }
        }
    };
}

macro_rules! assert_false {
    ($expr:expr) => {
        {
            let val = $expr;
            if val {
                panic!("Assertion failed: {} is not false (value: {:?})", 
                      stringify!($expr), val);
            }
        }
    };
}

macro_rules! assert_contains {
    ($container:expr, $item:expr) => {
        {
            let container = &$container;
            let item = &$item;
            if !container.contains(item) {
                panic!("Assertion failed: {:?} does not contain {:?}", 
                      container, item);
            }
        }
    };
}

// Benchmark macro
macro_rules! benchmark {
    ($name:expr, $iterations:expr, $code:block) => {
        {
            println!("\n⏱️  Benchmarking: {} ({} iterations)", $name, $iterations);
            
            let start = std::time::Instant::now();
            
            for _ in 0..$iterations {
                let _ = $code;
            }
            
            let total_duration = start.elapsed();
            let avg_duration = total_duration / $iterations;
            
            println!("  Total time: {:?}", total_duration);
            println!("  Average time per iteration: {:?}", avg_duration);
            println!("  Iterations per second: {:.0}", 
                    1_000_000_000.0 / avg_duration.as_nanos() as f64);
            
            (total_duration, avg_duration)
        }
    };
}

// Property-based testing macro
macro_rules! property_test {
    ($name:ident: $property:expr, $generator:expr, $iterations:expr) => {
        fn $name() {
            println!("\n🔍 Property test: {}", stringify!($name));
            
            let mut passed = 0;
            let mut failed = 0;
            
            for i in 0..$iterations {
                let test_value = $generator(i);
                
                if $property(test_value) {
                    passed += 1;
                } else {
                    failed += 1;
                    println!("  ❌ Property failed for input: {:?}", test_value);
                }
            }
            
            println!("  Results: {}/{} passed ({:.1}%)", 
                    passed, $iterations, (passed as f64 / $iterations as f64) * 100.0);
            
            if failed == 0 {
                println!("  ✅ Property holds for all test cases");
            } else {
                println!("  ❌ Property failed for {} test cases", failed);
            }
        }
    };
}

// Example functions to test
fn add(a: i32, b: i32) -> i32 {
    a + b
}

fn multiply(a: i32, b: i32) -> i32 {
    a * b
}

fn is_even(n: i32) -> bool {
    n % 2 == 0
}

fn factorial(n: u32) -> u32 {
    match n {
        0 | 1 => 1,
        _ => n * factorial(n - 1),
    }
}

// Create test suites using the framework
test_framework!(
    suite math_tests {
        test addition_works {
            assert_eq_msg!(add(2, 3), 5);
            assert_eq_msg!(add(-1, 1), 0);
            assert_eq_msg!(add(0, 0), 0);
        }
        
        test multiplication_works {
            assert_eq_msg!(multiply(3, 4), 12);
            assert_eq_msg!(multiply(-2, 5), -10);
            assert_eq_msg!(multiply(0, 100), 0);
        }
        
        test even_number_detection {
            assert_true!(is_even(2));
            assert_true!(is_even(0));
            assert_false!(is_even(1));
            assert_false!(is_even(3));
        }
        
        test factorial_calculation {
            assert_eq_msg!(factorial(0), 1);
            assert_eq_msg!(factorial(1), 1);
            assert_eq_msg!(factorial(5), 120);
        }
        
        test string_operations {
            let text = "Hello, World!";
            assert_contains!(text, "Hello");
            assert_contains!(text, "World");
            assert_true!(text.len() > 10);
        }
        
        test intentional_failure {
            // This test will fail to demonstrate error handling
            assert_eq_msg!(add(2, 2), 5); // This should fail
        }
    }
);

// Property-based tests
property_test!(addition_commutative: 
    |x: i32| add(x, 5) == add(5, x),
    |i| (i % 100) as i32,
    50
);

property_test!(multiplication_by_zero:
    |x: i32| multiply(x, 0) == 0,
    |i| (i % 1000) as i32 - 500,
    100
);

fn testing_framework_demo() {
    println!("\n=== Testing Framework Demo ===");
    
    // Run the test suite
    math_tests::run_all_tests();
    
    // Run property-based tests
    addition_commutative();
    multiplication_by_zero();
    
    // Benchmark examples
    benchmark!("Simple addition", 1000, {
        add(42, 58)
    });
    
    benchmark!("Factorial calculation", 100, {
        factorial(10)
    });
    
    benchmark!("String operations", 1000, {
        let s = "Hello, World!".to_string();
        s.to_uppercase()
    });
    
    println!("\n🎯 Testing framework features demonstrated:");
    println!("  ✅ Custom assertion macros");
    println!("  ✅ Test suite organization");
    println!("  ✅ Error handling and reporting");
    println!("  ✅ Property-based testing");
    println!("  ✅ Benchmarking capabilities");
    println!("  ✅ Comprehensive test statistics");
}

testing_framework_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What's the difference between declarative and procedural macros?
2. How does macro hygiene work in Rust?
3. What are the main benefits of using macros?
4. How do you debug macro expansion issues?
5. What are token trees and how do macros use them?
6. When should you use macros vs functions?
7. How do you handle repetition in macro patterns?
8. What are the limitations of Rust macros?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do macros enable powerful abstractions in Rust?**
2. **What are the trade-offs between macros and regular functions?**
3. **How can macros improve code maintainability and reduce errors?**
4. **What challenges does metaprogramming introduce?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Unsafe Rust & FFI

In our next lesson, you'll learn about:
- Unsafe Rust and raw pointers
- Foreign Function Interface (FFI)
- Calling C code from Rust
- Safety invariants and best practices

### How This Connects
Macros are essential for:
- Reducing boilerplate code
- Creating domain-specific languages
- Compile-time code generation
- Building powerful abstractions

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Create declarative macros with macro_rules!?
- [ ] Understand macro hygiene and scoping?
- [ ] Use pattern matching in macros effectively?
- [ ] Handle repetition and optional elements?
- [ ] Debug macro expansion issues?
- [ ] Choose between macros and functions appropriately?
- [ ] Create domain-specific languages with macros?

If you checked all boxes, excellent! You've mastered Rust's macro system.

---

**🎉 Incredible Achievement!** You now understand how to create powerful code generation tools with Rust macros!